In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

In [2]:
class FullModel(torch.nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 5)
        self.fc2 = torch.nn.Linear(5, 5)
        self.fc3 = torch.nn.Linear(5, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [3]:
def func(x):
    return np.power(x, 1) * np.power(np.sin(1.0 / 3.0 * x), 2)

def avg_l2_diff(y1, y2):
    return np.average(np.power(y1-y2, 2))

In [4]:
def train(model, optimizer, criterion, epoch):
    model.train()
    
    running_loss  = torch.tensor(0.0)

    for batch_idx, current_batch in enumerate(train_loader):     
        inp, current_batch_y = current_batch[0],        current_batch[1]

        optimizer.zero_grad()
        output = model(inp)
        gndtruth = current_batch_y

        loss = criterion(output, gndtruth)
        loss.backward()
        optimizer.step()
        running_loss  += loss.item()

    running_loss  = running_loss  / len(train_loader)
    
    if epoch % 20 == 0:
        print("Epoch: {}, Average loss: {:15.8f}".format(epoch, running_loss))

In [5]:
l2_diff_tot = []
for seed in np.arange(20):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True)
    
    model = FullModel()
    #x_train = np.arange(-2.4, 3.2, 0.8)
    #x_train = np.arange(-2.4, 2.8, 0.4)
    x_train = np.arange(-2.4, 2.6, 0.2)
    #x_train = np.arange(-2.4, 2.5, 0.1)
    #x_train = np.arange(-2.4, 2.45, 0.05)
   
    x_train_close = x_train + 0.0002
    #########################################################
    x_train = np.concatenate((x_train, x_train_close), axis=0)
    ##################################
    #x_train = x_train_close
    ##################################

    y_train = func(x_train)
    x_train_torch = torch.from_numpy(x_train).float()
    y_train_torch = torch.from_numpy(y_train).float()
    train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(1, 1000):
        train(model, optimizer, criterion, epoch)
        
    x_pred_np = np.arange(-2.5, 2.5, 0.0001).reshape(-1, 1)
    x_pred = torch.from_numpy(x_pred_np).float()
    y_true = func(x_pred_np)
    y_pred = model(x_pred)
    x_np = x_pred.numpy().reshape(-1)
    y_pred_np = y_pred.detach().numpy().reshape(-1)
    y_true_np = y_true.reshape(-1)
    l2_diff = avg_l2_diff(y_pred_np, y_true_np)
    l2_diff_tot.append(l2_diff)

Epoch: 20, Average loss:      0.24011497
Epoch: 40, Average loss:      0.15137163
Epoch: 60, Average loss:      0.10148881
Epoch: 80, Average loss:      0.07991924
Epoch: 100, Average loss:      0.07057678
Epoch: 120, Average loss:      0.06534673
Epoch: 140, Average loss:      0.06155558
Epoch: 160, Average loss:      0.05867315
Epoch: 180, Average loss:      0.05591463
Epoch: 200, Average loss:      0.05350813
Epoch: 220, Average loss:      0.05101611
Epoch: 240, Average loss:      0.04877408
Epoch: 260, Average loss:      0.04662221
Epoch: 280, Average loss:      0.04460065
Epoch: 300, Average loss:      0.04265369
Epoch: 320, Average loss:      0.04099711
Epoch: 340, Average loss:      0.03937257
Epoch: 360, Average loss:      0.03754946
Epoch: 380, Average loss:      0.03598245
Epoch: 400, Average loss:      0.03456532
Epoch: 420, Average loss:      0.03329560
Epoch: 440, Average loss:      0.03210960
Epoch: 460, Average loss:      0.03099591
Epoch: 480, Average loss:      0.02984

In [6]:
print(l2_diff_tot)

[0.013226124818201034, 0.009894742457601457, 0.010842830210310017, 0.014123337668578076, 0.0171655306955941, 0.005020050117426284, 0.018731548893838188, 0.025738702209780896, 0.010660220694274998, 0.012655573100036227, 0.013547418460063783, 0.0086748667852175, 0.042866845036323986, 0.006746288260515021, 0.005327283867998342, 0.014187123868507187, 0.006540643515524068, 0.006031003415014038, 0.006301080365983321, 0.008046857422174637]
